In [1]:
import pandas as pd
import numpy as np

In [2]:
# Path
metadata_path = 'metadata.csv'
adj_matrix_path = 'Adjacency Matrix.csv'

# Ouput Path
adj_matrix_output_path = 'Adjacency_Matrix_Preprocess.csv'

In [3]:
meta_df = pd.read_csv(metadata_path)
adj_matrix_df = pd.read_csv(adj_matrix_path)

In [4]:
meta_df.head()

,Node,Pos_X,Pos_Y,Destination
0,1,879,4924,Parabol Gate
1,2,1790,4924,Material Base
2,3,2106,4924,Material Base
3,4,2918,4924,Viet-Duc Building
4,5,3944,4924,TQB-Library


In [5]:
adj_matrix_df.head()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3.0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
adj_matrix_df.tail()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
31,32.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
32,33.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,34.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,35.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
35,NaN,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35


In [7]:
adj_matrix_df = adj_matrix_df.iloc[:adj_matrix_df.shape[0]-1, :]
adj_matrix_df.tail()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
30,31.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
31,32.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
32,33.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,34.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,35.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
print(adj_matrix_df.shape)

(35, 36)


In [9]:
row_count = adj_matrix_df.shape[0]
column_count = adj_matrix_df.shape[1]
print(row_count)
print(column_count)

35
36


In [10]:
# Test extract data from meta_df
node_test = meta_df[meta_df['Node']==1]
print(node_test)
print(type(node_test['Pos_X']))
print(node_test['Pos_X'].values[0])
print(node_test['Pos_Y'].values[0])

   Node  Pos_X  Pos_Y   Destination
0     1    879   4924  Parabol Gate
<class 'pandas.core.series.Series'>
879
4924


In [11]:
#Utils

# Compute Weight - Euclidean Distance
def compute_norm_2(point1 : tuple, point2 : tuple):
    dx = (point2[0] - point1[0])**2
    dy = (point2[1] - point1[1])**2
    return (dx + dy)**(1/2)

def compute_weight(node_num_1, node_num_2, _meta_data):
    node_1 = _meta_data[_meta_data['Node']==node_num_1]
    node_2 = _meta_data[_meta_data['Node']==node_num_2]
    
    node_1_x = node_1['Pos_X'].values[0]
    node_1_y = node_1['Pos_Y'].values[0]
    node_2_x = node_2['Pos_X'].values[0]
    node_2_y = node_2['Pos_Y'].values[0]
    
    return compute_norm_2((node_1_x, node_1_y), (node_2_x, node_2_y))

# test
print(compute_weight(1, 2, meta_df))

911.0


In [12]:
for i in range(row_count):
    for j in range(1, column_count):
        if adj_matrix_df.iloc[i, j] == 1:
            adj_matrix_df.iloc[i, j] = compute_weight(i+1, j, meta_df)

adj_matrix_df.head()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.0,911.0,0.0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2.0,911.0,0.0,316.0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1076.078529
2,3.0,0.0,316.0,0.0,812.0,0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,4.0,0.0,0.0,812.0,0.0,1026,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1069.105233,0.000000
4,5.0,0.0,0.0,0.0,1026.0,0,1320.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [13]:
adj_matrix_df_sum = np.sum(adj_matrix_df.iloc[:, 1:].to_numpy())
print(adj_matrix_df_sum)
adj_matrix_df_min = np.min(adj_matrix_df.iloc[:, 1:].to_numpy())
adj_matrix_df_max = np.max(adj_matrix_df.iloc[:, 1:].to_numpy())
print("MIN: {}".format(adj_matrix_df_min))
print("MAX: {}".format(adj_matrix_df_max))

83374.21479812678
MIN: 0.0
MAX: 2855.0112083843032


In [14]:
multi_coef = 1
for i in range(row_count):
    for j in range(1, column_count):
        if adj_matrix_df.iloc[i, j] != 0:
            adj_matrix_df.iloc[i, j] = (adj_matrix_df.iloc[i, j]*multi_coef - adj_matrix_df_min)/(adj_matrix_df_max - adj_matrix_df_min)
        else:
            adj_matrix_df.iloc[i, j] = adj_matrix_df.iloc[i, j]

adj_matrix_df.head()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.000000,0.319088,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2.0,0.319088,0.000000,0.110683,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.376909
2,3.0,0.000000,0.110683,0.000000,0.284412,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,4.0,0.000000,0.000000,0.284412,0.000000,0.359368,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.374466,0.000000
4,5.0,0.000000,0.000000,0.000000,0.359368,0.000000,0.462345,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [15]:
adj_matrix_df.head()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.000000,0.319088,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2.0,0.319088,0.000000,0.110683,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.376909
2,3.0,0.000000,0.110683,0.000000,0.284412,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,4.0,0.000000,0.000000,0.284412,0.000000,0.359368,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.374466,0.000000
4,5.0,0.000000,0.000000,0.000000,0.359368,0.000000,0.462345,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [16]:
adj_matrix_df.tail()

,Node,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
30,31.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.761311,0.0,0.0,0.0,0.0,0.000000,0.000000
31,32.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.729619,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
32,33.0,0.0,0.000000,0.0,0.000000,0.0,0.532093,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
33,34.0,0.0,0.000000,0.0,0.374466,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.395803
34,35.0,0.0,0.376909,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.395803,0.000000


In [17]:
adj_matrix_df.iloc[:, 1:].describe()

,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,30,31,32,33,34,35
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,...,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.037688,0.023048,0.023660,0.029093,0.023478,0.050267,0.028361,0.033985,0.030653,0.016615,...,0.006325,0.013586,0.044518,0.050090,0.010530,0.021752,0.020846,0.015203,0.022008,0.022077
std,0.175910,0.083584,0.087622,0.097111,0.097561,0.151400,0.118650,0.117592,0.107659,0.076674,...,0.037418,0.056547,0.162117,0.172873,0.062295,0.128685,0.123328,0.089940,0.090738,0.091017
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,0.376909,0.432999,0.374466,0.462345,0.580383,0.580383,0.527494,0.527494,0.432999,...,0.221365,0.270284,0.729619,0.761311,0.368540,0.761311,0.729619,0.532093,0.395803,0.395803


In [18]:
adj_matrix_df.iloc[:, 1:].to_csv(adj_matrix_output_path, 
                          index = False,
                          header = False
                         )